# Convolutional Neural Network

### Importing the libraries

In [1]:
import tensorflow as tf
from keras.preprocessing.image import ImageDataGenerator

2023-06-28 00:00:33.540402: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
tf.__version__

'2.12.0'

## Part 1 - Data Preprocessing

### Preprocessing the Training set

In [3]:
train_datagen = ImageDataGenerator(
        rescale=1./255,   # Feature scaling – gets all pixel values in e[0,1]
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True)
training_set = train_datagen.flow_from_directory(
        'dataset/training_set',
        target_size=(64, 64),
        batch_size=32,
        class_mode='binary')  # we either have cat or dog (binary)


Found 8024 images belonging to 2 classes.


### Preprocessing the Test set

In [4]:
test_datagen = ImageDataGenerator(rescale=1./255) # no other transformations. we feed it untouched images. just feature scale.
test_set = test_datagen.flow_from_directory(
        'dataset/test_set',
        target_size=(64, 64),
        batch_size=32,
        class_mode='binary')


Found 2006 images belonging to 2 classes.


## Part 2 - Building the CNN

### Initialising the CNN

In [5]:
cnn = tf.keras.models.Sequential()

### Step 1 - Convolution

In [6]:
cnn.add(tf.keras.layers.Conv2D(filters=32, kernel_size=3, activation='relu', input_shape=[64,64,3]))

### Step 2 - Pooling

In [7]:
# Max pooling
cnn.add(tf.keras.layers.MaxPool2D(pool_size=2, strides=2))

### Adding a second convolutional layer + pooling

In [8]:
cnn.add(tf.keras.layers.Conv2D(filters=32, kernel_size=3, activation='relu'))
# Max pooling
cnn.add(tf.keras.layers.MaxPool2D(pool_size=2, strides=2))

### Step 3 - Flattening

In [9]:
cnn.add(tf.keras.layers.Flatten())

### Step 4 - Full Connection

In [10]:
cnn.add(tf.keras.layers.Dense(units=128, activation='relu'))

### Step 5 - Output Layer

In [11]:
cnn.add(tf.keras.layers.Dense(1, activation='sigmoid')) # 1 neuron because we're doing binary classification

## Part 3 - Training the CNN

### Compiling the CNN

In [12]:
cnn.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

### Training the CNN on the Training set and evaluating it on the Test set

In [13]:
import sys
from PIL import Image
sys.modules['Image'] = Image 
cnn.fit(x=training_set, validation_data=test_set, epochs=25)

Epoch 1/25


2023-06-28 00:01:11.359861: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype int32
	 [[{{node Placeholder/_0}}]]


251/251 [==============================] - ETA: 0s - loss: 0.6731 - accuracy: 0.5825

2023-06-28 00:01:50.659408: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype int32
	 [[{{node Placeholder/_0}}]]


251/251 [==============================] - 46s 181ms/step - loss: 0.6731 - accuracy: 0.5825 - val_loss: 0.6060 - val_accuracy: 0.6625
Epoch 2/25
251/251 [==============================] - 45s 180ms/step - loss: 0.6106 - accuracy: 0.6694 - val_loss: 0.5559 - val_accuracy: 0.7208
Epoch 3/25
251/251 [==============================] - 44s 174ms/step - loss: 0.5602 - accuracy: 0.7136 - val_loss: 0.5365 - val_accuracy: 0.7418
Epoch 4/25
251/251 [==============================] - 47s 189ms/step - loss: 0.5274 - accuracy: 0.7404 - val_loss: 0.5514 - val_accuracy: 0.7303
Epoch 5/25
251/251 [==============================] - 43s 173ms/step - loss: 0.5007 - accuracy: 0.7517 - val_loss: 0.4959 - val_accuracy: 0.7557
Epoch 6/25
251/251 [==============================] - 42s 168ms/step - loss: 0.4760 - accuracy: 0.7666 - val_loss: 0.4762 - val_accuracy: 0.7822
Epoch 7/25
251/251 [==============================] - 44s 175ms/step - loss: 0.4672 - accuracy: 0.7783 - val_loss: 0.5192 - val_accuracy: 0.7

## Part 4 - Making a single prediction

In [18]:
import numpy as np
from keras.utils import load_img, img_to_array
test_image = load_img('dataset/single_prediction/cat_or_dog_2.jpg', target_size=(64,64))
test_image = img_to_array(test_image)
test_image = np.expand_dims(test_image, axis=0)
result = cnn.predict(test_image)
training_set.class_indices
if result[0][0] == 1: # first and only element in batch -> first and only prediction
  prediction = 'dog'
else:
  prediction = 'cat'

1/1 [==============================] - 0s 33ms/step


In [19]:
print(prediction)

cat
